In [1]:
!KERAS_BACKEND=tensorflow python -c "from keras import backend"

Using TensorFlow backend.


In [2]:
import os
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from PIL import Image

from glob import glob

Using TensorFlow backend.


In [3]:
!ls

123gogo.sh
bottleneck_fc_model.h5
bottleneck_features_train.npy
bottleneck_features_validation.npy
Cats_2_bottleneck.ipynb
Cats_3_gpu2.ipynb
Cats_gpu2.ipynb
cats-weights-improvement-03-0.66.hdf5
cats-weights-improvement-12-0.63.hdf5
Dogs_4
Dogs_4_gpu2.ipynb
dogs4_predictions.csv
Dogs_alltime_gpu2.ipynb
Dogs_alltime.zip
DogsVCats_gpu2.ipynb
dogsVcats-weights-improvement-04-0.58.hdf5
jupyter.log
__MACOSX
nohup.out
Nora_gpu2-Copy1.ipynb
Nora_gpu2.ipynb
Photo_Rand_gpu2.ipynb
Predicting.ipynb
rajDogs4-weights-improvement-09-0.61.hdf5
rajDogs4-weights-improvement-19-0.62.hdf5
rajDogs4-weights-improvement-20-0.63.hdf5
rajDogs4-weights-improvement-26-0.63.hdf5
rajphoto-weights-improvement-15-0.51.hdf5
vgg16_weights_tf_dim_ordering_tf_kernels.h5
weights-improvement-35-0.40.hdf5


In [4]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'Dogs_alltime/train'
validation_data_dir = 'Dogs_alltime/validation'

In [5]:
# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)

# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=16,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')

Found 28632 images belonging to 2 classes.
Found 12376 images belonging to 2 classes.


In [6]:
model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [5]:
model_vgg = Sequential()
model_vgg.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height,3)))
model_vgg.add(Convolution2D(64, (3, 3), activation='relu', name='conv1_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(64, (3, 3), activation='relu', name='conv1_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(128, (3, 3), activation='relu', name='conv2_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(128, (3, 3), activation='relu', name='conv2_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, (3, 3), activation='relu', name='conv3_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, (3, 3), activation='relu', name='conv3_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, (3, 3), activation='relu', name='conv3_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, (3, 3), activation='relu', name='conv4_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, (3, 3), activation='relu', name='conv4_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, (3, 3), activation='relu', name='conv4_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, (3, 3), activation='relu', name='conv5_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, (3, 3), activation='relu', name='conv5_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, (3, 3), activation='relu', name='conv5_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(Flatten())
model_vgg.add(Dense(64))
model_vgg.add(Activation('relu'))
model_vgg.add(Dropout(0.5))
model_vgg.add(Dense(1))
model_vgg.add(Activation('sigmoid'))

In [7]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [8]:
nb_epoch = 30
nb_train_samples = 28632
nb_validation_samples = 12376

In [9]:
filepath="rajDogsalltime-weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [10]:
model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples//16, #40661 // batch_size
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples//16, #15502//batch_size
        callbacks=callbacks_list)

/usr/local/lib/python2.7/dist-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=111, epochs=30, callbacks=[<keras.ca..., validation_steps=773)`
  '` call to the Keras 2 API: ' + signature)


Epoch 1/30
111/111 [==============================] - 306s - loss: 0.7096 - acc: 0.5220 - val_loss: 0.6858 - val_acc: 0.5544
Epoch 2/30
111/111 [==============================] - 306s - loss: 0.6963 - acc: 0.5366 - val_loss: 0.6866 - val_acc: 0.5670
Epoch 3/30
111/111 [==============================] - 306s - loss: 0.6848 - acc: 0.5681 - val_loss: 0.6680 - val_acc: 0.6142
Epoch 4/30
111/111 [==============================] - 306s - loss: 0.6808 - acc: 0.5850 - val_loss: 0.6645 - val_acc: 0.6067
Epoch 5/30
111/111 [==============================] - 306s - loss: 0.6700 - acc: 0.6098 - val_loss: 0.6629 - val_acc: 0.5949
Epoch 6/30
111/111 [==============================] - 306s - loss: 0.6694 - acc: 0.5929 - val_loss: 0.7118 - val_acc: 0.5616
Epoch 7/30
111/111 [==============================] - 306s - loss: 0.6670 - acc: 0.6075 - val_loss: 0.7066 - val_acc: 0.5376
Epoch 8/30
111/111 [==============================] - 306s - loss: 0.6652 - acc: 0.6064 - val_loss: 0.6340 - val_acc: 0.6306


In [11]:
model.evaluate_generator(validation_generator, nb_validation_samples)

[0.71692228614006859, 0.63441694292021777]

In [12]:
spots = nb_validation_samples/32

In [14]:
val_predictions = model.predict_generator(validation_generator,(spots+1))

In [15]:
preds = pd.DataFrame(val_predictions)
files = pd.DataFrame(validation_generator.filenames)
predictions = files.join(preds,lsuffix=0, rsuffix= nb_validation_samples)
predictions.columns = ['path','probability']
predictions.head()

,path,probability
0,bad/ee0bd4dc469b3fc89514860bdd39b42a3bbf4905.jpg,0.244680
1,bad/091457a94f16171448dfb8a62f73ee17c6d404b1.jpg,0.240777
2,bad/fc341ef33566b1050f2c2f1768b4196fb6dc8dd7.jpg,0.456035
3,bad/9202fb6d37cda2f4652755e0e0171f649cdc4f91.jpg,0.628914
4,bad/3850b6fb6ca17bbead06b62b8dfe94eac65eeece.jpg,0.305365


In [16]:
predictions.to_csv('dogs_alltime_predictions.csv')